In [1]:
#!pip install tensorflow tensorflow-recommenders
# 
from typing import Dict, Text, Union

import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_hub as hub
import tensorflow_text
import pandas as pd
import re
import gc

import importlib

USE_MOD_URL = 'https://tfhub.dev/google/universal-sentence-encoder/4'
EVENTS_PATH = "/home/thusitha/work/projects/recommendation_take_home/data/sample_1k_users.csv"

In [2]:
events_dt = pd.read_csv(EVENTS_PATH)
print(events_dt.shape)
events_dt

(3143221, 6)


,user_id,item_id,behavior_type,user_location_hash,item_category,time
0,90795949,402391768,1,94h6dlp,3046,2014-12-09 21
1,90795949,116725988,1,94h6dkp,3487,2014-12-17 23
2,90795949,230564881,1,NaN,11489,2014-11-28 00
3,90795949,327767037,3,NaN,12855,2014-12-09 23
4,90795949,79851069,1,94h6dkc,5037,2014-12-10 12
...,...,...,...,...,...,...
3143216,84420232,58229968,1,95qqs15,8141,2014-12-12 07
3143217,84420232,357613414,2,95qqs1o,6513,2014-12-06 12
3143218,84420232,194792004,1,95qqsut,8141,2014-12-12 07
3143219,84420232,60277228,1,95qqs1a,8141,2014-12-12 07


In [3]:
#events_ts_ds = tf.data.Dataset.from_tensor_slices(dict(events_dt[["resume_id", "job_id"]]))
events_ts_ds = tf.data.Dataset.from_tensor_slices(dict(events_dt[["user_id", "item_id"]]))
print(len(events_ts_ds))

events_ds = events_ts_ds.map(lambda x: {
    "user_id": x["user_id"],
    "item_id": x["item_id"]
})

items_ds = events_ts_ds.map(lambda x: {
    "item_id": x["item_id"]
})



3143221


2022-05-02 08:28:51.289457: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 08:28:51.294195: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 08:28:51.294573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 08:28:51.295311: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
unique_user_ids = np.unique(events_dt["user_id"])
user_ids_vocabulary.adapt(unique_user_ids)

In [5]:
item_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
unique_item_ids = np.unique(events_dt["item_id"])
item_ids_vocabulary.adapt(unique_item_ids)


In [7]:
print(user_ids_vocabulary.vocabulary_size(), item_ids_vocabulary.vocabulary_size())

906 832916


In [6]:
tf.random.set_seed(42)
shuffled_data = events_ds.shuffle(len(events_ds))

n_train = int(len(events_ds)*0.9)
n_test = len(events_ds) - n_train

#n_train = int(len(events_ds)*0.10)
#n_test = int(len(events_ds)*0.10)

train = shuffled_data.take(n_train)
test = shuffled_data.skip(n_train).take(n_test)

print(len(train), len(test))

2828898 314323


In [11]:

import models.recomender_model
importlib.reload(models.recomender_model)
from models.recomender_model import DEFAULTS, RecommenderModel



In [19]:

model = None
gc.collect()
model = RecommenderModel(user_ids_vocabulary, item_ids_vocabulary, items_ds, dense_layers = None) 

Consider rewriting this model with the Functional API.


In [13]:
cached_train = train.batch(DEFAULTS["batch_size"]).cache()
cached_test = test.batch(DEFAULTS["batch_size"]).cache()

In [14]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))
model.fit(cached_train, epochs=1)


Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
691/691 [==============================] - 22s 27ms/step - top_10_categorical_accuracy: 0.0000e+00 - top_100_categorical_accuracy: 0.0000e+00 - loss: 33748.3864 - regularization_loss: 0.0000e+00 - total_loss: 33748.3864


In [14]:
model.evaluate(cached_test, return_dict=True)

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
30/39 [======================>.......] - ETA: 3:08 - top_10_categorical_accuracy: 0.0031 - top_100_categorical_accuracy: 0.0288 - loss: 32590.1562 - regularization_loss: 0.0000e+00 - total_loss: 32590.1562

In [ ]:
# # Create a retrieval model.

# # Use brute-force search to set up retrieval using the trained representations.
# index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# index.index_from_dataset(
#     test.batch(100).map(lambda x:x["job_id"]).map(lambda _id: (_id, model.jobs_model(_id))))



In [ ]:
# # Get some recommendations.
# _, titles = index(np.array([2]))
# print(f"Top 3 recommendations for user 1: {titles[0, :3]}")



In [16]:
# import tensorflow_hub as hub
# import tensorflow as tf
# import tensorflow_text

# USE_MOD_URL = 'https://tfhub.dev/google/universal-sentence-encoder/4'

# embed = hub.load(URL)